# How to Send a LPL Modeling Code to the LPLServer using Julia

> *Author*: Tony Hürlimann, 12. Jun 2019,  <a href="mailto:tony.huerlimann@unifr.ch">Contact Me</a>

LPL is a mathematical modeling language that allows one to formulate large linear, integer and non-linear models close to the mathematical notation. The formulation code can then be sent to a solver (like **[Gurobi](http://www.gurobi.com)** or other solvers) to solve the model. LPL contains many additional features that extend it to be a complete programming language. For more information on LPL see **[Short Intro to LPL](http://www.virtual-optima.com/download/docs/lplflash.pdf)**.

This notebook explains how to send a LPL code (and a complete optimization model) to the LPLServer using the POST protocoll with Julia. Due to security restrictions some file access instructions are not allowed. Note also that the running time is limited.

This notebook explains several methods to send a LPL model and retrieve various results. The general method is explained in 4 steps.

**Step 1**: Import the ``HTTP`` library to Julia.

In [ ]:
using HTTP

**Step 2**: Define the url for the LPLServer and define the LPL code. Both variables are simple strings.

In [ ]:
url   = "http://lpl.unifr.ch/lpl/LPLS"
model = "model abc; set i:=1..100; Write{i}('%3d %7d\n',i,i^3); end"

**Note**: The model is enclosed within apostophes ("), since single quotes (') are used inside the model code. The LPL code above prints the numbers from 1 to 100 together with its cubic values.

**Step 3**: Send a POST request and store the return value in a variable *r* :

In [ ]:
r = HTTP.request("POST",url,body=model)

The retrun value stores in its ``text`` property the ``write`` output of the model (the NOM-file as it is called in the LPL world -- that is, everything the ``Write`` statements have produced).

**Step 4**: Get the output from the return value.

In [ ]:
print(String(r.body))

In [ ]:
r.status

The code ``r.status`` is the http code returned. 200 means "success".

# Return the Solution, not the Writes

Instead of returning the output of the ``Write`` statements, one also can return to variables and their values of a mathematical model. All one needs to do is to send an additional parameter in the url string, that is : ``id=1``

In [ ]:
url1 = "http://lpl.unifr.ch/lpl/LPLS?id=1"

Of course, we need now a mathematical model that contains variables and constraints, that is, a complete mathematical model. Here the model is somewhat trivial. It is:
$$ \max\ x \ \ \text{ subject to }\  x+y=10$$
The solution is trival, and can be found manually, it is simply :
$$ x=10\ ,\ y=0 $$

In [ ]:
model1 = "model a; variable x;y; constraint A:x+y=10; maximize obj:x; end"

In [ ]:
r = HTTP.request("POST",url1,body=model1)

In [ ]:
print(String(r.body))

We can use indexed variables as well. A small -- also trivial -- model is this :

$$ \begin{array}{ll}
       \text{set} & I =\{1,\ldots,10\} \\
       \text{variables} & 1 \leq x_{i} \leq 10\ \ (i\in I)\ ,\ y \geq 0 \\
       \max & x_1   \\
       \text{subject to} & x_i + y = 20 \quad\text{for all } i\in I  \\
  \end{array} $$

It is easy to see that the solution is :
$$ x_i=10 \ \ (i\in I)\ ,\ y=10 $$ 

In [ ]:
model2 = """
model a;
  set i:=1..10; 
  variable x{i} [1..10]; y; 
  constraint A{i} : x+y = 20; 
  maximize obj : x[1]; 
end
"""

In [ ]:
r = HTTP.request("POST",url1,body=model2)

In [ ]:
print(String(r.body))

Next we show how to solve a larger linear model consisting of 1000 constraints and 2000 variables. The model is as follows:
  $$ \begin{array}{ll}
       \max & \mathbf c \cdot \mathbf x   \\
       \textrm{subject to} & \mathbf A\cdot \mathbf x \leq \mathbf b  \\
                           & \mathbf x \geq 0
  \end{array} $$
  The data for the matrix $A$, and the vectors $b$ and $c$ are generated in a random way. $x$ are the variables. Note that """....""" is the pythonic way to define a multiline string

In [ ]:
modelLP = """
model Lp2000 "A larger 2000x1000 general LP";
  set  m :=  [1..1000];    n :=  [1..2000];
  parameter
    A{m,n} := if(Rnd(0,1)<0.02 , Rnd(0,60));
    c{n}   := if(Rnd(0,1)<0.87 , Rnd(0,9));
    b{m}   := if(Rnd(0,1)<0.87 , Rnd(10,70000));
  variable  x{n};
  constraint R{m}:  sum{n} A*x <= b;
  maximize Obj:  sum{n}  c*x;
end
"""

In [ ]:
r=HTTP.request("POST",url1,body=modelLP)

In [ ]:
print(String(r.body))

Note that the variables not shown in the list are zero.

# A Model from the Williams book

This is the same model as shown at ** [Gurobi webinar Python I](http://www.gurobi.com/resources/seminars-and-videos/python-I-webinar) **

In [ ]:
urlm = "http://lpl.unifr.ch/lpl/LPLS"

modelm = """
model Will03 "Factory planning I"; 
set
  p:=[Prod1 Prod2 Prod3 Prod4 Prod5 Prod6 Prod7] "prod";
  m:=[grinder vDrill hDrill borer planer]  "mach";
  t:=[Jan Feb Mar Apr May Jun]   "period";
parameter  profit{p} := [10 6 8 4 11 9 3];
  time{m,p} "machine time table" :=[.5 .7 . . .3 .2 .5, 
    .1 .2 . .3 . .6 . , .2 . .8 . . . .6 ,
    .05 .03 . .07 .1 . .08 , . . .01 . .05 . .05]; 
  down{t,m} "number of machines down" := [ 
   (Jan,grinder) 1  (Feb,hDrill)  2  (Mar,borer)  1
   (Apr,vDrill)  1  (May,grinder) 1  (May,vDrill) 1
   (Jun,planer)  1  (Jun,hDrill)  1]; 
  qMach{m} "nr. of machines available" := [4 2 3 1 1]; 
  upper{t,p} "market limitation of sells" := [500 1000 
   300 300 800 200 100, 600 500 200 0 400 300 150, 300
   600 0 0 500 400 100, 200 300 400 500 200 0 100, 0
   100 500 100 1000 300 0, 500 500 100 300 1100 500 60];
  storeCost := 0.5;   storeCapacity := 100;
  endStock  := 50;    hoursMonth    := 2*8*24; 
variable
  manu{t,p}            "quantity manufactured";
  held{t,p}            "quantity stored";
  sell{t,p} [0..upper] "quantity sold"; 
constraint
  Balance{t,p}: held[t-1,p] + manu = sell + held;
  endstore{p}: held[#t,p] = endStock;
  storecapa{t,p|t<#t}: held <= storeCapacity;
  capa{t,m}: sum{p} time*manu <= hoursMonth*(qMach-down); 
maximize Profit: sum{t,p} (profit*sell - storeCost*held);

Writep(Profit,manu,held,sell);
end
"""

r=HTTP.request("POST",urlm,body=modelm)

print(String(r.body))



# Return a SVG Graph instead

LPL can generate svg graphs with its **Draw library**. The following model is an example of a model that 
generates a svg graph.

In [ ]:
model3 = """
model DrawFilters "Draw Filters";
  Draw.Scale(2,2);
  Draw.DefFilter('f', 9,11,1, 4);                            // 9=feGaussianBlur
  Draw.DefFilter('f',13,1 ,2, 4,4);                          //13=feOffset
  Draw.DefFilter('f',14,1 ,3, 5,.75,20,-2,Rgb(187,187,187)); //14=feSpecularLighting
  Draw.DefFilter('f',18,-5000,-10000,20000);                 //18=fePointLight
  Draw.DefFilter('f', 4,3,11,3,2);                           // 4=feComposite
  Draw.DefFilter('f', 4,10,3,4,6,0,1,1,0);                   // 4=feComposite
  Draw.DefFilter('f',11);                                    //11=feMerge
  Draw.DefFilter('f',24,2);                                  //24=feMergeNode
  Draw.DefFilter('f',24,4);                                  //24=feMergeNode
    
  Draw.Rect(1,20,198,85,Rgb(136,136,136),4);
  //Draw.Group('#f');
  Draw.Path('#f','M50,90 C0,90 0,30 50,30 L150,30 C200,30 200,90 150,90 z',-1,5,2*10);    
  Draw.Path('#f','M60,80 C30,80 30,40 60,40 L140,40 C170,40 170,80 140,80 z',5);
  //Draw.EndGroup();
  Draw.Text('#f','LPL',62,76,2*45);
  
  Draw.Rect(1,125,198,60,Rgb(136,136,136),4);
  Draw.Circle('#f',30,155,25,2);
  Draw.Rect('#f',70,130,50,50,3);
  Draw.Path('#f','M130,155 l50,-25 l0,50 z',4);  
end
"""

In this case, the parameter ``id=2`` must be sent to the LPLServer :

In [ ]:
url2 = "http://lpl.unifr.ch/lpl/LPLS?id=2"

In [70]:
r = HTTP.request("POST",url2,body=model3)

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Server: Apache-Coyote/1.1
Set-Cookie: JSESSIONID=A79DA6B38A0AD9000F9F707E0CCFD94D; Path=/lpl/; HttpOnly
Content-Length: 2180
Date: Fri, 07 Jun 2019 12:01:16 GMT

&#60;?xml version="1.0" encoding="ISO-8859-1" standalone="no" ?&#62;
&#60;!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 20010904//EN"
"http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd"&#62;
&#60;svg xmlns="http://www.w3.org/2000/svg"
  xmlns:xlink="http://www.w3.org/1999/xlink"
  width="410" height="340"  viewBox="-5 35 410 340"&#62;

  &#60;defs&#62;
    &#60;filter id="f"&#62;
      &#60;feGaussianBlur in="SourceAlpha" result="1" stdDeviation="4"/&#62;
      &#60;feOffset in="1" result="2" dx="4" dy="4"/&#62;
      &#60;feSpecularLighting in="1" result="3" surfaceScale="5" specularConstant="0.75" specularExponent="20" lighting-color="#BBBBBB"&#62;
        &#60;fePointLight  x="-5000" y="-10000" z="20000"/&#62;
      &#60;/feSpecularLighting&#62;
      &#60;feComposite in="3" in2="So

In [ ]:
r.body

In [ ]:
print(String(r.body))

The returned text contains the html code for '<' and '>', which is '``&#60;``' and '``&#62;``'. One needs to replace them and let's copy the result in a string with the name ``text`` :

In [ ]:
text=String(r.body)
#text=replace(text,"&#60;" => "<")
#text=replace(text,"&#62;" => ">")

In [ ]:
print(text)


Next the string ``text`` is written to a file ``Output.svg``.

In [ ]:
text_file = open("Output.svg", "w")
write(text_file,String(r.body))
close(text_file)

Finally let's load the SVG and display modules and then display the svg file, that is the svg file that the LPL code generated initially on the LPLServer :

In [ ]:
f= open("Output.svg")
display("image/svg+xml", readstring(f))

This ends the short tutorial about sending and running LPL code to the LPLServer. 